**Setting up the environment**

In [1]:
%%capture
files = !ls
files = [f.split("  ") for f in files][0]

isFRIQML = 'fri_qml' in files
isFRIQMLPath = isFRIQML and "setup.py" in files

# Clone the entire repo. Only run once!
if not isFRIQML:
  !git clone -l -s https://github.com/znajob/fri_qml.git fri_qml

if not isFRIQMLPath:
  %cd fri_qml

!git pull
!pip install -e .

In [2]:
# MAIN IMPORTS
import pennylane as qml
from pennylane import numpy as np
from friqml.visualisation import plot_quantum_state, plot_histogram
from friqml.utils import eps, random_state_normalized, random_state_unnormalized
from functools import partial
from tqdm.notebook import tqdm

When solving the exercises refer to the [PennyLane documentation](https://pennylane.readthedocs.io/en/stable/).

## Many-body problem

### Exercise 1

The **Ising model** is a basic model of statistical mechanics that explains a lot about how quantum optimizers work. Its energy is described by its Hamiltonian:

\begin{align}
H=-\sum_{i=1}^{n-1} J_{i,i+1} \sigma_i \sigma_{i+1} - \sum_{i=1}^n h_i \sigma_i.
\end{align}

Write a function `h_ising` that calculates this energy for a linear chain of spins. The function takes three arguments: `J`, `h`, and `sigma`, corresponding to the coupling strengths, the onsite field at each site, and the specific spin configuration.

In [ ]:
from friqml.utils import eps

In [ ]:
def h_ising(J, h, sigma):
    s = 0
    for i in range(len(J)):
        s -= J[i] * sigma[i] * sigma[i+1]
    for i in range(len(h)):
        s -= h[i] * sigma[i]
    return s

In [9]:
# TESTS
J = [1.0, -1.0]
h = [0.5, 0.5, 0.4]
sigma = [+1, -1, +1]
print( abs(h_ising(J, h, sigma)+0.4) < eps )
J = [-1.0, 0.5, 0.9]
h = [4, 0.2, 0.4, 0.7]
sigma = [+1, -1, -1, -1]
print(abs(h_ising(J, h, sigma)+5.1) < eps)

True
True


### Exercise 2
Finding the lowest possible energy of the Ising Hamiltonian is a combinatorial optimization problem, and it is a known NP-hard problem. Many heuristic methods have been invented to tackle the problem. One of them is simulated annealing. It is implemented in [dimod](https://docs.dwavequantum.com/en/latest/ocean/api_ref_dimod/index.html#dimod).

1.   Write a function `random_antiferromagnetic_ising` with one argument `n` that creates a random antiferromagnetic model on `n` sites with no external field $h=0$ and returns the variables `J` and `h`. Then write a function `ising_solution` that (given given `J` and `h`) iterates over all possible configurations, calculates the corresponding energies and returns the configuration with the smallest energy and a list of all energies in ascending order.

2.   Create the same antiferromagnetic model in dimod with the same `J` and `h`. Keep in mind that dimod uses a plus and not a minus sign in the Hamiltonian, so the sign of your couplings should be reversed. Store the model in an object called `model`, which should be a [`BinaryQuadraticModel`](https://docs.dwavequantum.com/en/latest/ocean/api_ref_dimod/models.html#dimod.binary.BinaryQuadraticModel). Sample the solution space a hundred times and write the response in an object called [`sampleset`](https://docs.dwavequantum.com/en/latest/ocean/api_ref_dimod/sampler_composites.html#dimod.reference.samplers.SimulatedAnnealingSampler).




In [ ]:
#SOLUTION
import dimod


In [ ]:
def dimod_qubo_ising(J, h):
    a = -h
    b = np.diag(-J, 1)
    qbm = dimod.BinaryQuadraticModel(a, b, "SPIN")
    return qbm

def random_antiferromagnetic_ising(n):
    J = []
    h = []
    for i in range(n-1):
        J.append(-np.random.sample())
        h.append(0)
    h.append(0)

    return np.array(J), np.array(h)

def ising_solution(J, h):
    min_energy = np.inf
    energies_all = []
    sigma_best = None
    for i in range(2**len(h)):
        #sigma = [2*int(x)-1 for x in bin(i)[2:]]
        sigma = [2*int(x)-1 for x in bin(i)[2:].zfill(len(h))]
        energy = h_ising(J, h, sigma)
        if energy < min_energy:
          sigma_best = sigma
          min_energy = energy
        energies_all.append(energy)
    return sigma_best, sorted(energies_all)


In [ ]:
# TESTS
n=3
J,h = random_antiferromagnetic_ising(n)
sigma,elist=ising_solution(J,h)
#print(sigma)
#print(elist)
model = dimod_qubo_ising(J,h)
#print(model)
sampleset = dimod.SimulatedAnnealingSampler().sample(model,num_reads=100)
#print(sampleset.first.sample)

print(all([sigma[i]*sigma[i+1] == -1 for i, _ in enumerate(J)]))
print(len(sampleset) == 100)
sample = sampleset.first.sample
print(all([sample[i]*sample[i+1] == -1 for i, _ in enumerate(J)]))

True
True
True


### Exercise 3
A quantum formulation of the classical Ising model is written in terms of the Pauli matrix $\sigma^{\rm z}=\begin{pmatrix}1 & 0\\ 0& -1\end{pmatrix}$. The Hamiltonian is

\begin{align}
H= -\sum_{<i,j>}J_{i,j}\sigma^{\rm z}_i\sigma^{\rm z}_j-\sum_jh_j\sigma^{\rm z}_j.
\end{align}
Write a function `classical_ising` that receives `J` and `h` as arguments and returns a matrix representing the quantum Hamiltonian of the 1D classical Ising model. Then calculate the spectrum of the Hamiltonian and check that it corresponds to all possible values of the function `h_ising`. Also check that the Hamiltonian is diagonal.

In [ ]:
def sz(i, n):
    assert i < n, "i has to be smaller than n"
    return np.kron(np.kron(np.eye(2**(i)), np.array([[1, 0], [0, -1]])), np.eye(2**(n-i-1)))


def classical_ising(J, h):
    n = len(J)
    H = np.zeros((2**n, 2**n))
    sz = np.array([[1,0], [0,-1]])
    for i in range(n-1):
      H -= J[i] * np.kron(np.eye(2**(i)), np.kron(sz, np.eye(2**(n-i)))) @ np.kron(np.eye(2**(i+1)), np.kron(sz, np.eye(2**(n-i-2))))
    for i in range(n):
      H -= h[i] * np.kron(np.eye(2**(i)), np.kron(sz, np.eye(2**(n-i))))

In [72]:
# TESTS
n=7
J,h = random_antiferromagnetic_ising(n)
sigma,elist=ising_solution(J,h)
H = classical_ising(J,h)
es = np.linalg.eigvalsh(H)
print(max(abs(es- elist))<eps)
print(np.max(abs(H-np.diag(np.diag(H))))==0.0)

ValueError: operands could not be broadcast together with shapes (64,64) (128,128) (64,64) 

### Exercise 4
A model consisting only of $\sigma^{\rm z}$ terms is classical since all operators in the Hamiltonian commute. Moreover, the only non-zero elements are on the diagonal. The simplest "quantum" model is obtained by adding a local field in the $x$ direction

\begin{align}
H= -\sum_{<i,j>}J_{i,j}\sigma^{\rm z}_i\sigma^{\rm z}_j-\sum_j(h^{\rm z}_j\sigma^{\rm z}_j+ h^{\rm x}_j\sigma^{\rm x}_j).
\end{align}
Create a function `transverse_ising` that receives `J`, `hz` and `hx` as arguments and returns the Hamiltonian of the transverse field Ising model. Check that the Hamiltonian is not diagonal and that the ground state (the state with the smallest energy) is an entangled state.


In [ ]:

from friqml.solutions.entanglement import is_entangled
from friqml.utils import eps, sz, sx

In [ ]:
def transverse_ising(J, hz, hx):
    n = len(hz)
    H = classical_ising(J, hz)
    for i in range(n):
        H -= hx[i]*sx(i, n)
    return H

In [ ]:
n=4
J = np.random.rand(n-1)
hz = np.random.rand(n)
hx = np.random.rand(n)
H = transverse_ising(J,hz,hx)
if n<=2:
  print(H)

In [ ]:
# TESTS
print(np.max(abs(H-np.diag(np.diag(H))))>eps)
val,vec = np.linalg.eigh(H)
print(is_entangled(vec[0]))

True
True


## Gate model of quantum computing

### Exercise 1
Quantum computers typically initialize their qubit registers in the $|0\rangle$ state. This means that if there is any particular state we would like to work with, first we have to figure out how to create that state with a circuit. Some states are easier to prepare than others. If you are just given a random vector, say, $\begin{bmatrix}0.36\\  0.8704\end{bmatrix}$, it is not easy to figure out how to prepare it. In fact, the very purpose of quantum computing is to prepare a probability distribution of interest, that is, a state. So in some ways, generic state preparation is as hard as or equivalent to quantum computation. On the other hand, some states are easy to prepare; for instance, the state $\frac{-|0\rangle + |1\rangle}{\sqrt{2}}$. Create a template `haar_random_unitary` represents a Haar random unitary, i.e. prepares a Haar random state by applying the unitary on $|0\rangle$. For more in depth undersanding of the Haar measure consult the PennyLane demo [Understanding the Haar Measure](https://pennylane.ai/qml/demos/tutorial_haar_measure.html)

In [ ]:
# DEVICE
dev = qml.device('default.qubit', wires=1, shots=None)

In [ ]:
from friqml.utils import get_vector
from scipy.stats import rv_continuous

In [ ]:
class sin_prob_dist(rv_continuous):
    def _pdf(self, theta):
        # The 0.5 is so that the distribution is normalized
        return 0.5 * np.sin(theta)


# Samples of theta should be drawn from between 0 and pi
sin_sampler = sin_prob_dist(a=0, b=np.pi)


def haar_random_unitary(wires):
    # Sample phi and omega as normal
    phi, omega = 2 * np.pi * np.random.uniform(size=2)
    theta = sin_sampler.rvs(size=1)  # Sample theta from our new distribution
    qml.Rot(phi, theta, omega, wires=wires)


In [ ]:
@qml.qnode(dev)
def circuit():
  haar_random_unitary(wires=0)
  return qml.state()

In [ ]:
'''def bloch_sphere_coordinates(alpha, beta):
    """
    Compute the Bloch sphere coordinates (theta, phi) and Cartesian coordinates (x, y, z) of a qubit given its state coefficients.

    Parameters:
        alpha (complex): The amplitude of the |0> state.
        beta (complex): The amplitude of the |1> state.

    Returns:
        tuple: (theta, phi, x, y, z) in radians and Cartesian coordinates.
    """
    # Normalize the state
    norm = np.sqrt(abs(alpha)**2 + abs(beta)**2)
    alpha /= norm
    beta /= norm

    # Compute theta
    theta = 2 * np.arccos(abs(alpha))

    # Compute phi (azimuthal angle)
    phi = np.angle(beta) - np.angle(alpha)

    # Ensure phi is in range [0, 2pi)
    phi = (phi + 2 * np.pi) % (2 * np.pi)

    # Compute Cartesian coordinates
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    return [x, y, z], [theta, phi]'''

In [ ]:
# TESTS
nsamp = 10000
thetas = np.zeros(nsamp)
phis = np.zeros(nsamp)
for i in tqdm(range(nsamp)):
  psi = circuit()[0]
  a=psi[0]/abs(psi[0])
  _,angles = get_vector(psi[0]/a,psi[1]/a)
  thetas[i] = np.real(angles[0])
  phis[i] = np.real(angles[1])

cths = np.cos(thetas)
cthdens,_=np.histogram(cths,density=True,bins=10)
phidens,_=np.histogram(phis,density=True,bins=10)
print((abs(cthdens-1/(max(cths)-min(cths)))<0.05).all())
print((abs(phidens-1/(max(phis)-min(phis)))<0.05).all())

  0%|          | 0/10000 [00:00<?, ?it/s]

True
True


In [ ]:
abs(cthdens-1/(max(cths)-min(cths)))

tensor([0.01800306, 0.00900153, 0.01850314, 0.01450246, 0.00350059,
        0.0165028 , 0.01100187, 0.02700459, 0.00250042, 0.01150195], requires_grad=True)

### Exercise 2
Create a function `circuit` that prepares the state $\frac{1}{\sqrt{2}}(|000\rangle+|111\rangle)$ and returns 100 samples in the computational basis.

In [ ]:
# DEVICE
nsample=100
dev = qml.device('default.qubit', wires=3, shots=nsample)

In [ ]:
# SOLUTION
from friqml.solutions.gate_model import e2_circuit
circuit = qml.qnode(dev)(e2_circuit)

In [ ]:
def e2_circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    return qml.sample()

In [ ]:
# TESTS
samples = circuit()
nzero = len([1 for x in samples if x[0]==0 and x[1]==0 and x[2]==0])
none = len([1 for x in samples if x[0]==1 and x[1]==1 and x[2]==1])
print(nzero+none==nsample)
print((nzero-none)/nsample<0.1)

True
False


## Adiabatic quantum computing

### Exercise 1
In an adiabatic process, conditions change slowly enough for the system to adapt to the new configuration. We can start from some Hamiltonian $H_0$ and slowly change it to some other Hamiltonian $H_1$, for instance, on a linear schedule: $H(t) = (1-t) H_0 + t H_1$. The speed of change heavily depends on the energy gap, that is, the difference between the ground state energy and the first excited state of all Hamiltonians $H(t)$, $t\in[0,1]$.

It is easy to craft a Hamiltonian where this gap is small, so the speed limit has to be low. If you take a classical Ising model with coupling strengths on vastly different scales, that is what you get, for example

\begin{align}
H_{\rm small-gap}=-1000\sigma^Z_1\sigma^Z_2-0.1\sigma^Z_2\sigma^Z_3-0.5\sigma^Z_1
\end{align}
Write a function `small_gap_hamiltonian` that returns a matrix representing the above Hamiltonian. Calculate the spectral gap, i.e. the difference between two smallest eigenvalues of the returned matrix.


For comparison write also a function `big_gap_hamiltonian` that returns the matrix representation of the Hamiltonian

\begin{align}
H_{\rm big-gap} = -\sigma^{\rm x}_1-\sigma^{\rm x}_2-\sigma^{\rm x}_3
\end{align}


Remember that since you have three qubits, the $\sigma^Z_1\sigma^Z_2$ operator, for instance, actually means $\sigma^Z\otimes\sigma^Z\otimes\mathbb{1}$.

In [ ]:
def small_gap_hamiltonian():
    H = -1000*sz(0, 3)@sz(1, 3) - 0.1*sz(1, 3)@sz(2, 3) - 0.5*sz(0, 3)
    return H


def big_gap_hamiltonian():
    H = -sx(0, 3)-sx(1, 3)-sx(2, 3)
    return H

In [ ]:
H = small_gap_hamiltonian()
e = np.linalg.eigvalsh(H)
gap = e[1]-e[0]
print(np.isclose(gap,0.2))

H = big_gap_hamiltonian()
e = np.linalg.eigvalsh(H)
gap = e[1]-e[0]
print(np.isclose(gap,2))

True
True


### Exercise 2
 On a real quantum annealing device, we drop the stringent theoretical requirements of following the adiabatic pathway and we repeat the transition over and over again. Then we choose the lowest energy solution as our optimum.

The classical 'simulator' for a quantum annealer is some heuristic solver of combinatorial optimization, for instance, simulated annealing. Use the dimod package to implement the Hamiltonian with a small gap: $H_1=-1000\sigma^Z_1\sigma^Z_2-0.1\sigma^Z_2\sigma^Z_3-0.5\sigma^Z_1$. Write a function `small_gap_model` that returns a `BinaryQuadraticModel` object for the "small gap" Hamiltonian.

In [ ]:
# SOLUTION
import dimod

In [ ]:
def small_gap_model():
    a = np.zeros(3)
    a[0] = -0.5

    b = np.zeros([3, 3])
    b[0, 1] = -1000
    b[1, 2] = -0.1

    qbm = dimod.BinaryQuadraticModel(a, b, "SPIN")
    return qbm

In [ ]:
# TESTS
model = small_gap_model()
sampler = dimod.SimulatedAnnealingSampler()
response = sampler.sample(model, num_reads=10)
print(np.isclose(response.first.energy, -1000.6))

True


**Note:** Unlike in the case of a simple system, you often do not get the ground state:

In [ ]:
print([solution.energy for solution in response.data()])

[np.float64(-1000.6), np.float64(-1000.6), np.float64(-1000.6), np.float64(-1000.6), np.float64(-1000.6), np.float64(-999.6), np.float64(-999.6), np.float64(-999.6), np.float64(-999.6), np.float64(-999.6)]
